In [122]:
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

In [2]:
train_df = pd.read_csv(Path('Resources/2019loans.csv'))
test_df = pd.read_csv(Path('Resources/2020Q1loans.csv'))

In [5]:
train_df.head()
# len(train_df)
# train_df.isnull().sum()
# train_drop = train_df.dropna()
# len(train_drop)

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,target
0,7000.0,0.1894,256.38,MORTGAGE,75000.0,Not Verified,n,28.62,0.0,2.0,...,87.5,0.0,0.0,352260.0,62666.0,35000.0,10000.0,N,N,low_risk
1,40000.0,0.1614,975.71,MORTGAGE,102000.0,Source Verified,n,11.72,2.0,0.0,...,0.0,0.0,0.0,294664.0,109911.0,9000.0,71044.0,N,N,low_risk
2,11000.0,0.2055,294.81,RENT,45000.0,Verified,n,37.25,1.0,3.0,...,7.7,0.0,0.0,92228.0,36007.0,33000.0,46328.0,N,N,low_risk
3,4000.0,0.1612,140.87,MORTGAGE,38000.0,Not Verified,n,42.89,1.0,0.0,...,100.0,0.0,0.0,284273.0,52236.0,13500.0,52017.0,N,N,low_risk
4,14000.0,0.1797,505.93,MORTGAGE,43000.0,Source Verified,n,22.16,1.0,0.0,...,25.0,0.0,0.0,120280.0,88147.0,33300.0,78680.0,N,N,low_risk


In [6]:
for c in train_df.columns:
    print(c)

loan_amnt
int_rate
installment
home_ownership
annual_inc
verification_status
pymnt_plan
dti
delinq_2yrs
inq_last_6mths
open_acc
pub_rec
revol_bal
total_acc
initial_list_status
out_prncp
out_prncp_inv
total_pymnt
total_pymnt_inv
total_rec_prncp
total_rec_int
total_rec_late_fee
recoveries
collection_recovery_fee
last_pymnt_amnt
collections_12_mths_ex_med
policy_code
application_type
acc_now_delinq
tot_coll_amt
tot_cur_bal
open_acc_6m
open_act_il
open_il_12m
open_il_24m
mths_since_rcnt_il
total_bal_il
il_util
open_rv_12m
open_rv_24m
max_bal_bc
all_util
total_rev_hi_lim
inq_fi
total_cu_tl
inq_last_12m
acc_open_past_24mths
avg_cur_bal
bc_open_to_buy
bc_util
chargeoff_within_12_mths
delinq_amnt
mo_sin_old_il_acct
mo_sin_old_rev_tl_op
mo_sin_rcnt_rev_tl_op
mo_sin_rcnt_tl
mort_acc
mths_since_recent_bc
mths_since_recent_inq
num_accts_ever_120_pd
num_actv_bc_tl
num_actv_rev_tl
num_bc_sats
num_bc_tl
num_il_tl
num_op_rev_tl
num_rev_accts
num_rev_tl_bal_gt_0
num_sats
num_tl_120dpd_2m
num_tl_30dpd
n

In [9]:
# Checking each column for any nulls.  If no nulls are found, rows are condensed.
for column in train_df.columns:
    if train_df[column].isnull().sum() >0:
        print(f'\nColumn {column} has {train_df[column].isnull().sum()} null values')
    else: print(0, end = ', ')

0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [7]:
test_df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,target
0,40000.0,0.1033,856.40,RENT,128700.0,Source Verified,n,12.47,0.0,1.0,...,57.1,0.0,0.0,63915.0,49510.0,49400.0,14515.0,Y,N,low_risk
1,24450.0,0.1430,572.72,MORTGAGE,44574.0,Not Verified,n,15.05,0.0,1.0,...,0.0,0.0,0.0,136425.0,19439.0,15500.0,18925.0,N,N,low_risk
2,13500.0,0.1430,316.23,OWN,60000.0,Not Verified,n,28.72,0.0,0.0,...,0.0,0.0,0.0,82124.0,65000.0,5400.0,61724.0,Y,N,low_risk
3,10625.0,0.1774,268.31,RENT,60000.0,Verified,n,15.70,0.0,4.0,...,20.0,0.0,0.0,54855.0,50335.0,23200.0,26255.0,N,N,low_risk
4,6375.0,0.1862,232.46,RENT,60000.0,Source Verified,n,35.50,0.0,0.0,...,75.0,0.0,0.0,90445.0,56541.0,15300.0,72345.0,N,N,low_risk


In [10]:
# Checking each column for any nulls.  If no nulls are found, rows are condensed.
for column in test_df.columns:
    if train_df[column].isnull().sum() >0:
        print(f'\nColumn {column} has {train_df[column].isnull().sum()} null values')
    else: print(0, end = ', ')

0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [11]:
len(test_df.columns)

84

In [12]:
# Convert categorical data to numeric and separate target feature for training data
X = train_df.drop('target', axis = 1)
X.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,7000.0,0.1894,256.38,MORTGAGE,75000.0,Not Verified,n,28.62,0.0,2.0,...,100.0,87.5,0.0,0.0,352260.0,62666.0,35000.0,10000.0,N,N
1,40000.0,0.1614,975.71,MORTGAGE,102000.0,Source Verified,n,11.72,2.0,0.0,...,85.7,0.0,0.0,0.0,294664.0,109911.0,9000.0,71044.0,N,N
2,11000.0,0.2055,294.81,RENT,45000.0,Verified,n,37.25,1.0,3.0,...,94.3,7.7,0.0,0.0,92228.0,36007.0,33000.0,46328.0,N,N
3,4000.0,0.1612,140.87,MORTGAGE,38000.0,Not Verified,n,42.89,1.0,0.0,...,92.9,100.0,0.0,0.0,284273.0,52236.0,13500.0,52017.0,N,N
4,14000.0,0.1797,505.93,MORTGAGE,43000.0,Source Verified,n,22.16,1.0,0.0,...,84.1,25.0,0.0,0.0,120280.0,88147.0,33300.0,78680.0,N,N


In [22]:
X_train = pd.get_dummies(X)
X_train

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,hardship_flag_Y,debt_settlement_flag_N,debt_settlement_flag_Y
0,7000.0,0.1894,256.38,75000.0,28.62,0.0,2.0,20.0,0.0,40414.0,...,0,1,1,0,1,0,1,0,1,0
1,40000.0,0.1614,975.71,102000.0,11.72,2.0,0.0,10.0,0.0,43531.0,...,0,1,0,1,1,0,1,0,1,0
2,11000.0,0.2055,294.81,45000.0,37.25,1.0,3.0,23.0,0.0,8242.0,...,1,1,0,1,1,0,1,0,1,0
3,4000.0,0.1612,140.87,38000.0,42.89,1.0,0.0,7.0,0.0,12767.0,...,0,1,0,1,0,1,1,0,1,0
4,14000.0,0.1797,505.93,43000.0,22.16,1.0,0.0,22.0,0.0,11182.0,...,0,1,0,1,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12175,19975.0,0.2565,801.09,28000.0,28.42,0.0,0.0,15.0,0.0,9679.0,...,0,1,0,1,1,0,1,0,1,0
12176,15000.0,0.1774,540.34,50000.0,23.43,4.0,0.0,16.0,0.0,3193.0,...,1,1,0,1,1,0,1,0,1,0
12177,3600.0,0.1862,131.28,60000.0,28.80,0.0,1.0,14.0,0.0,11804.0,...,0,1,0,1,1,0,1,0,1,0
12178,15000.0,0.0881,475.68,62000.0,11.44,0.0,0.0,5.0,0.0,157.0,...,0,1,0,1,0,1,1,0,1,0


In [138]:
train_df['target'].unique()

array(['low_risk', 'high_risk'], dtype=object)

In [95]:
y = pd.get_dummies(train_df['target'])
y_train = y[['high_risk']]
y_train

,high_risk
0,0
1,0
2,0
3,0
4,0
...,...
12175,1
12176,1
12177,1
12178,1


In [96]:
# Convert categorical data to numeric and separate target feature for testing data
X = test_df.drop('target', axis = 1)
X_test = pd.get_dummies(X)

y = pd.get_dummies(test_df['target'])
y_test = y[['high_risk']]
y_test.head()

,high_risk
0,0
1,0
2,0
3,0
4,0


In [106]:
train_list = X_train.columns.to_list()
test_list = X_test.columns.to_list()
# print(train_list)


# check for differences between column lists
# print(list(set(train_list).symmetric_difference(set(test_list))))

print("missing from train_list: ",list(set(test_list) - set(train_list)))
print("missing from test_list: ", list(set(train_list) - set(test_list)))

print(f'train column count: {len(train_list)}')
print(f'test column count:  {len(test_list)}')

# for i in range(len(train_list)):
#     print(f'{train_list[i]:>35}  {test_list[i]}')

missing from train_list:  []
missing from test_list:  ['debt_settlement_flag_Y']
train column count: 92
test column count:  91


In [114]:
# add missing dummy variables to testing set
X_test['debt_settlement_flag_Y'] = 0
X_test.head(2)

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,hardship_flag_Y,debt_settlement_flag_N,debt_settlement_flag_Y
0,40000.0,0.1033,856.40,128700.0,12.47,0.0,1.0,8.0,0.0,38113.0,...,0,1,0,1,1,0,0,1,1,0
1,24450.0,0.1430,572.72,44574.0,15.05,0.0,1.0,6.0,0.0,1665.0,...,0,1,0,1,1,0,1,0,1,0


### Prediction:
I suspect the scaled random forest classifier will perform best because of a mixture of discrete and non-integer numbers.

In [127]:
def confusion_score(y_true, y_pred):
    [[TP, FN],[FP,TN]] = confusion_matrix(y_true, y_pred)

    accuracy = (TP + TN) / (TP + FP + TN + FN) # (111 + 128) / (111 + 5 + 128 + 6)
    print(f"Accuracy: {accuracy.round(2)}")
    precision = TP / (TP + FP)
    print(f'Precision: {precision.round(2)}')
    sensitivity = TP / (TP + FN)
    print(f'Sensitivity: {sensitivity.round(2)}')
    specificity = TN / (TN + FP)
    print(f'Specificity: {specificity.round(2)}')

In [124]:
# Train the Logistic Regression model on the unscaled data and print the model score
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

C:\Users\Rob\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Training Data Score: 0.65311986863711
Testing Data Score: 0.5072309655465759


C:\Users\Rob\anaconda3\envs\PythonData\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [132]:
# Logistic Regression Confusion Matrix
y_true = y_test
y_pred = classifier.predict(X_test)
print('Confusion Matrix:\n',confusion_matrix(y_true, y_pred), '\n')

confusion_score(y_true, y_pred)

Confusion Matrix:
 [[1675  676]
 [1641  710]] 

Accuracy: 0.5072309655465759
Precision: 0.5051266586248492
Sensitivity: 0.7124627817949809
Specificity: 0.301999149298171


In [141]:
# Train a Random Forest Classifier model and print the model score
clf = RandomForestClassifier(random_state=1, n_estimators=500).fit(X_train, y_train)
print(f'Training Score: {clf.score(X_train, y_train)}')
print(f'Testing Score: {clf.score(X_test, y_test)}')

C:\Users\Rob\AppData\Local\Temp/ipykernel_21948/3806078898.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf = RandomForestClassifier(random_state=1, n_estimators=500).fit(X_train, y_train)


Training Score: 1.0
Testing Score: 0.6482347937048064


In [142]:
# Random Forest Classifier Confusion Matrix
y_true = y_test
y_pred = clf.predict(X_test)
print('Confusion Matrix:\n',confusion_matrix(y_true, y_pred), '\n')

confusion_score(y_true, y_pred)

Confusion Matrix:
 [[1090 1261]
 [ 393 1958]] 

Accuracy: 0.6482347937048064
Precision: 0.7349966284558328
Sensitivity: 0.4636324968098681
Specificity: 0.8328370905997448


In [115]:
# Scale the data
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
# X_train_scaled

scaler2 = StandardScaler().fit(X_test)
X_test_scaled = scaler2.transform(X_test)
# X_test_scaled

In [139]:
# Train the Logistic Regression model on the scaled data and print the model score
classifier = LogisticRegression()
classifier.fit(X_train_scaled, y_train)
print(f"Training Data Score: {classifier.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test_scaled, y_test)}")

C:\Users\Rob\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Rob\anaconda3\envs\PythonData\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Training Data Score: 0.710919540229885
Testing Data Score: 0.6599319438536793


In [140]:
# Scaled Logistic Regression Confusion Matrix
y_true = y_test
y_pred = classifier.predict(X_test_scaled)
print('Confusion Matrix:\n',confusion_matrix(y_true, y_pred), '\n')

confusion_score(y_true, y_pred)

Confusion Matrix:
 [[1502  849]
 [ 750 1601]] 

Accuracy: 0.6599319438536793
Precision: 0.6669626998223801
Sensitivity: 0.6388770735857082
Specificity: 0.6809868141216504


In [143]:
# Train a Random Forest Classifier model on the scaled data and print the model score
clf = RandomForestClassifier(random_state=1, n_estimators=500).fit(X_train_scaled, y_train)
print(f'Training Score: {clf.score(X_train_scaled, y_train)}')
print(f'Testing Score: {clf.score(X_test_scaled, y_test)}')

C:\Users\Rob\AppData\Local\Temp/ipykernel_21948/4080388192.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf = RandomForestClassifier(random_state=1, n_estimators=500).fit(X_train_scaled, y_train)


Training Score: 1.0
Testing Score: 0.5680561463207146


In [145]:
# Random Forest Classifier Confusion Matrix
y_true = y_test
y_pred = clf.predict(X_test_scaled)
print('Confusion Matrix:\n',confusion_matrix(y_true, y_pred), '\n')

confusion_score(y_true, y_pred)

Confusion Matrix:
 [[ 492 1859]
 [ 172 2179]] 

Accuracy: 0.5680561463207146
Precision: 0.7409638554216867
Sensitivity: 0.20927264993619737
Specificity: 0.9268396427052318


### Conclusions  
Random Forest Classifier seems to be overfitting the training data.  It also performed better prior to scaling.  Overall, I would have to rate the Scaled Logistic Regression as the best overall model used in this analysis.

One thing to note is that since the test data isn't a random sample of the same dataset as the training set, it can be affected by external changes.  The big one would be the introduction of Covid-19, which wouldn't have been affecting any data prior to 2020.